## Challenge 4: Advanced RAG with Azure AI Document intelligence

Many documents in  real scenario, are not just text, they are a combination of text, images, tables, etc. In this step, you will create a more advanced RAG application able to deal with this kind of documents.
For this reason, you will use Azure AI Document Intelligence to extract the text, images, and tables from the documents and use them as input for the RAG model.

To achieve this, we will build on top of the langchain framework enhancing the `Document Loader` and `Text Splitters` to deal with images and tables.
In the code repositiory, you have already the enhanced version of the `Document Loader` and `Text Splitters` that you can use. They are included in two different python modules: `document_loader.py` and `text_splitters.py`.

You can now use these libraries to create your advanced RAG.

We provided already the libraries and the Environment variables required (you need just to populate them).

In [3]:
import sys, os, dotenv
dotenv.load_dotenv()
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))

# Setup environment

# OpenAI
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OPENAI_EMBEDDING")
# Azure Search
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
# Azure AI Document Intelligence
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
AZURE_DOCUMENT_INTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")
# Azure Blob Storage
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
AZURE_STORAGE_CONTAINER = os.getenv("AZURE_STORAGE_CONTAINER")
AZURE_STORAGE_FOLDER = os.getenv("AZURE_STORAGE_FOLDER")

# Import Libraries
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from azure.ai.documentintelligence.models import DocumentAnalysisFeature

# Custom Libraries
from its_a_rag.doc_intelligence import AzureAIDocumentIntelligenceLoader
from its_a_rag import ingestion


## Create the Vector store, the embeddings client and the OpenAI Chat client

Let's start creating the vector store and the embeddings client. Because we need a custom index to store the information in the way so that our retriever wil be able to get it, we have a custom function for that (create_multimodal_vectore_store).
For the OpenAI Chat client we will simply use the one offered by langchain framework as in the Step 3 of this notebook.

In [4]:
# Create the index for Azure Search store and Embedding
vector_store_multimodal, aoai_embeddings = ingestion.create_multimodal_vector_store(AZURE_SEARCH_INDEX, 
                                                                           AZURE_OPENAI_API_KEY, 
                                                                           AZURE_OPENAI_ENDPOINT,
                                                                           AZURE_OPENAI_API_VERSION,
                                                                           AZURE_OPENAI_EMBEDDING, 
                                                                           AZURE_SEARCH_ENDPOINT, 
                                                                           AZURE_SEARCH_API_KEY)

# Create the Azure OpenAI Chat Client
llm = AzureChatOpenAI(api_key = AZURE_OPENAI_API_KEY,  
                      api_version = "2024-06-01",
                      azure_endpoint = AZURE_OPENAI_ENDPOINT,
                      model= AZURE_OPENAI_DEPLOYMENT_NAME,
                      streaming=True)

## Index Phase

As always the first step is to index the documents:
the high level steps are:

- Set Folder Path: Assign the local folder path to the variable folder.
- List Files: Create a list of files in the specified folder.
- Get Full Paths: Convert the list of file names to their full paths.
- Iterate Over Files: Loop through each file in the list.
    - Extract File Name: Extract the file name from the full path (this is required for the document loader).
    - Load Document: Use AzureAIDocumentIntelligenceLoader to load the document with specified API credentials and settings (remember to use pre-built layout as model and the latest API version)
    - Split Document: Split the loaded document using a custom advanced text splitter.
    - Store Document: Add the processed documents to a multimodal vector store (using the add_documents method).

In [5]:
import os
# Get list of files in a local folder
folder = os.path.join(os.getcwd(), '../../../its-a-rag/data/fsi/pdf') # TODO: FIX ME
files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)) and f.startswith("2023")]
files = [os.path.join(folder, f) for f in files]
print(files)

['/workspaces/ch-hackathons/its-a-rag-solutions/challenges/challenge4/../../../its-a-rag/data/fsi/pdf/2023 FY AMZN.pdf', '/workspaces/ch-hackathons/its-a-rag-solutions/challenges/challenge4/../../../its-a-rag/data/fsi/pdf/2023 FY APPL.pdf', '/workspaces/ch-hackathons/its-a-rag-solutions/challenges/challenge4/../../../its-a-rag/data/fsi/pdf/2023 FY GOOGL.pdf', '/workspaces/ch-hackathons/its-a-rag-solutions/challenges/challenge4/../../../its-a-rag/data/fsi/pdf/2023 FY MSFT.pdf', '/workspaces/ch-hackathons/its-a-rag-solutions/challenges/challenge4/../../../its-a-rag/data/fsi/pdf/2023 FY NVDA.pdf']


In [21]:
# Index

# Index: Load files

# Get list of files in a local folder
folder = os.path.join(os.getcwd(), '../../../its-a-rag/data/fsi/pdf') # TODO: FIX ME
files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)) and f.startswith("2023")]
files = [os.path.join(folder, f) for f in files]

# For each file
for file in files:
    # Get the file name
    pdf_file_name = file.split("\\")[-1]
    # Index : Load the file and create a document
    print("Processing: ", file)
    loader = AzureAIDocumentIntelligenceLoader(file_path=file, 
                                           api_key = "822fb076b87d4f86b211a568d63ed1df", 
                                           api_endpoint = "https://docint-kbbxgc.cognitiveservices.azure.com/",
                                           api_model="prebuilt-layout",
                                           api_version="2024-10-01",
                                           analysis_features = [DocumentAnalysisFeature.OCR_HIGH_RESOLUTION])
    docs = loader.load()
    # Index : Split
    docs = ingestion.advanced_text_splitter(docs,pdf_file_name)
    # Index : Store
    vector_store_multimodal.add_documents(documents=docs)

Processing:  /workspaces/ch-hackathons/its-a-rag-solutions/challenges/challenge4/../../../its-a-rag/data/fsi/pdf/2023 FY AMZN.pdf


ResourceNotFoundError: (404) Resource not found
Code: 404
Message: Resource not found

In [18]:
loader.load()

ResourceNotFoundError: (404) Resource not found
Code: 404
Message: Resource not found

## Retrieve Phase

The next step is to create a retriever for the documents based on the user query.
You should use the following parameters:
- Search Type: Hybrid
- number of results: 5

In [4]:
# Retrieve
retriever_multimodal = vector_store_multimodal.as_retriever(search_type="hybrid", k=5)

## Generate Phase

The final step is to generate the answer using the RAG model.
We will create a Langchain chain with the following steps:
 - Retrieve the docs and get the image desciption if the doc matedata is an image (with get_image_description function - RunnableLambda), then pass the context and question (using RunnablePassthrough) to the next phase
 - Use the advanced multimodal Prompt function to append system messages, the context including the text, the image (if present) and the question - check RannableLambda method also here.
 - Use the OpenAI model to generate the answer
 - Parse the output and return the answer

In [5]:
# Generate

# RAG pipeline
chain_multimodal_rag = (
    {
        "context": retriever_multimodal | RunnableLambda(ingestion.get_image_description),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(ingestion.multimodal_prompt)
    | llm
    | StrOutputParser()
)

## Test the Solution

You can test the solution by providing a question and checking the answer generated by the RAG model (invoke the Langchain chain).

Try to get answer for the following questions:


In [6]:
# Test the solution
print(chain_multimodal_rag.invoke("For intel, what is the GAAP and Non-GAAP for 2023 in percentage?"))
print(chain_multimodal_rag.invoke("What are the revenues of INTEL Comporation in the year 2000?"))
print(chain_multimodal_rag.invoke("What are the revenues and the operative margins of INTEL in 2022 and how it compares with the previous year?"))
print(chain_multimodal_rag.invoke("Can you compare and create a table with the revenue of INTEL in years 2019-2023?"))

For Intel in 2023:

- **GAAP Gross Margin**: 40.0%
- **Non-GAAP Gross Margin**: 43.6%

**Detailed Breakdown:**

| Measure                       | 2023      |
|-------------------------------|-----------|
| GAAP Gross Margin             | 40.0%     |
| Non-GAAP Gross Margin         | 43.6%     |

The GAAP gross margin decreased by 2.6 percentage points from 2022, and the Non-GAAP gross margin decreased by 3.7 percentage points from 2022.
I don't know.
In 2022, Intel's net revenue was $63,054 million, and the operating income was $2,334 million with an operating margin of 3.7%. 

Comparing it to 2021, the net revenue was $79,024 million, and the operating income was $19,456 million with an operating margin of 24.6%. 

Here's a concise comparison:

| Year | Net Revenue (in millions) | Operating Income (in millions) | Operating Margin (%) |
|------|----------------------------|-------------------------------|----------------------|
| 2022 | $63,054                   | $2,334               